In [1]:
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as utils

import helper

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numpy.linalg import norm

matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['text.latex.unicode'] = True

# For the notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
biwi = pd.read_csv('./data/train/biwi/biwi_hotel.txt', header = None,
                 names = ['frameNb','id', 'x','y'],delimiter=' ')
id_unique = np.unique(np.array(biwi['id']))

init = np.zeros(len(biwi)) 
biwi['Speed'] = init
biwi['Angle'] = init
biwi['Vx'] = init
biwi['Vy'] = init

In [4]:
pd.options.mode.chained_assignment = None ## Disable StettingWithCopy warning
c = 0
for i in id_unique:
    a = biwi[biwi['id']==i]
    ind = a.index
    a.index = range(len(a))
    dist = a['x'].iloc
    dist1 = a.loc[0:len(a)-2,'x':'y']
    dist1.index=range(len(dist1))
    dist2 = a.loc[1:,'x':'y']
    dist2.index=range(len(dist2))
    dist = dist2-dist1
    b = len(dist)
    if c < b:
        vector_speed = np.zeros((len(id_unique),b,2))
        c=b
b = 0
for i in id_unique:
    a = biwi[biwi['id']==i]
    ind = a.index
    a.index = range(len(a))
    dist = a['x'].iloc
    dist1 = a.loc[0:len(a)-2,'x':'y']
    dist1.index=range(len(dist1))
    dist2 = a.loc[1:,'x':'y']
    dist2.index=range(len(dist2))
    dist = dist2-dist1
    speed = np.array(np.sqrt(dist['x']**2+dist['y']**2)/0.4)
    biwi.loc[ind[1:],'Speed'] = speed
    angle=np.zeros(len(dist)-1)
    vx=np.zeros(len(dist))
    vy=np.zeros(len(dist))
    for j in range(len(dist)-1):
        if norm(dist.loc[j,:])==0 or norm(dist.loc[j+1,:])==0:
            angle[j]=0
        elif np.cross(dist.loc[j,:],dist.loc[j+1,:])/(norm(dist.loc[j,:])*norm(dist.loc[j+1,:]))>1:
            angle[j]=np.arcsin(1)
        else:
            angle[j]=np.arcsin(np.cross(dist.loc[j,:],dist.loc[j+1,:])/(norm(dist.loc[j,:])*norm(dist.loc[j+1,:])))
    
    for j in range(len(dist)):
        if j == 0:
            vx[j] = 0
            vy[j] = speed[j]
            vector_speed[b][j][0]=vx[j]
            vector_speed[b][j][1]=vy[j]
        else:
            vx[j] = speed[j]*np.sin(sum(angle[:j]))
            vy[j] = speed[j]*np.cos(sum(angle[:j]))
            vector_speed[b][j][0]=vx[j]
            vector_speed[b][j][1]=vy[j]
    
    biwi.loc[ind[2:],'Angle'] = angle
    biwi.loc[ind[1:],'Vx'] = vx
    biwi.loc[ind[1:],'Vy'] = vy
    b+=1

In [7]:
vector = np.zeros((len(vector_speed),2*len(vector_speed[0])))
x_train = np.zeros((len(vector_speed),20))
y_train = np.zeros((len(vector_speed),18))
for i in range(len(vector_speed)):
    vector[i][:] = np.reshape(vector_speed[i][:][:],2*19,'C')
for i in range(len(vector)):
    x_train[i][:] = vector[i][:20]
    y_train[i][:] = vector[i][20:38]

x_train,y_train = torch.from_numpy(x_train).type(torch.FloatTensor), torch.from_numpy(y_train).type(torch.FloatTensor)

traindataset = utils.TensorDataset(x_train, y_train)
trainloader = utils.DataLoader(traindataset, batch_size=2, shuffle=True)


In [8]:
class ConvNet(nn.Module):
    def __init__(self, n_input_channels=1, n_output=None):
        super().__init__()
        ################################################################################
        # TODO:                                                                        #
        # Define 2 or more different layers of the neural network                      #
        ################################################################################
        
        self.conv1 = nn.Conv1d(n_input_channels,8,5,padding=2)
        #self.conv1_bn = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True)
     
        self.fc1 = nn.Linear(19*2 * 8, 19*2 * 8)
        
        
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
    
    def forward(self, x):
        ################################################################################
        # TODO:                                                                        #
        # Set up the forward pass that the input data will go through.                 #
        # A good activation function betweent the layers is a ReLu function.           #
        #                                                                              #
        # Note that the output of the last convolution layer should be flattened       #
        # before being inputted to the fully connected layer. We can flatten           #
        # Variable `x` with `x.view`.                                                  #
        ################################################################################
        
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        x = x.view(-1, 19*2 * 40) # in order to reshape the tensor for as many columns we need
        x = self.fc1(x)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits)

In [9]:
net = ConvNet()
################################################################################
# TODO:                                                                        #
# Choose an Optimizer that will be used to minimize the loss function.         #
# Choose a critera that measures the loss                                      #
################################################################################

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-4)
epochs = 1
steps = 0
running_loss = 0
print_every = 50
for e in range(epochs):
    start = time.time()
    for images, labels in iter(trainloader):
        steps += 1
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        # HINT: Do not forget to transform the inputs and outputs into Variable        #
        # which pytorch uses.                                                          #
        ################################################################################
        inputs = Variable(images)
        targets = Variable(labels)
        optimizer.zero_grad()
        output = net.forward(inputs)
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        loss = criterion(output, targets)
        ################################################################################
        # TODO:                                                                        #
        # Run the training process                                                     #
        #                                                                              #
        # HINT: Calculate the gradient and move one step further                       #
        ################################################################################
        grad = loss.backward()
        optimizer.step()
        ################################################################################
        #                              END OF YOUR CODE                                #
        ################################################################################
        
        running_loss += loss.data[0]
        
        if steps % print_every == 0:
            stop = time.time()
            # Test accuracy
            accuracy = 0
            for ii, (images, labels) in enumerate(valloader):
                ################################################################################
                # TODO:                                                                        #
                # Calculate the accuracy                                                       #
                ################################################################################
                
                inputs = Variable(images, volatile=True)
                
                predicted = net.predict(inputs).data
                equality = (labels == predicted.max(1)[1])
                accuracy += equality.type_as(torch.FloatTensor()).mean()
                
                
                #im = Variable(images)
                #out = net.predict(im)
                #_,prediction = torch.max(out, 1)
                #pred_y = prediction.data.numpy().squeeze()
                #target_y = labels.numpy()
                #accuracy = np.mean(pred_y == target_y)
                #print(pred_y.shape,target_y.shape)
                
                
                ################################################################################
                #                              END OF YOUR CODE                                #
                ################################################################################
            
            print("Epoch: {}/{}..".format(e+1, epochs),
                  "Loss: {:.4f}..".format(running_loss/print_every),
                  "Test accuracy: {:.4f}..".format(accuracy/(ii+1)),
                  "{:.4f} s/batch".format((stop - start)/print_every)
                 )
            running_loss = 0
            start = time.time()

ValueError: Expected 3D tensor as input, got 2D tensor instead.

In [11]:
x_train


 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.0000  0.0000
          ...             ⋱             ...          
 0.0000  1.4372 -0.0261  ...   1.4368  0.1313  1.5607
 0.0000  1.5768 -0.0273  ...   1.4698 -0.3532  1.5185
 0.0000  1.3730 -0.1302  ...   1.5778 -0.3114  1.5441
[torch.FloatTensor of size 145x20]

In [52]:
biwi

,frameNb,id,x,y,Speed,Angle,Vx,Vy
0,0,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
1,10,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
2,20,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
3,30,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
4,40,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
5,50,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
6,60,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
7,70,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
8,80,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
9,90,5,-1.59,0.93,0.000000,0.000000,0.000000,0.000000
